In [23]:
import matplotlib.pyplot as plt  
import scipy
from scipy.stats import norm
from collections import Counter
from ipywidgets import interactive, IntSlider
import random

In [11]:
def read_sequence(filename):
    with open(filename, "r") as file: 
        sequence = []
        file_content = file.readlines()
        for i in range(len(file_content)):
            if file_content[i].startswith("@"):
                sequence.append(file_content[i+1])
    
    return sequence

In [18]:
def kmer_frequency(sequence, length):
    output_dict = {}
    
    for line in sequence:
        # Save the kmer frequency for each sequence
        line_dict = {}
        used_counts = 0
        skipped_counts = 0
        
        if len(line) >= length:
            for i in range(0,len(line)-length+1):
                fragment = line[i:i+length]
                count = line.count(fragment)
                line_dict[fragment] = count
                
            # Add the kmer frequency in each sequence to the total dictionary 
            for key in line_dict:
                if key in output_dict.keys():
                    output_dict[key] += line_dict[key]
                else:
                    output_dict[key] = line_dict[key]
                
            used_counts += 1

        else:
            print("Warning: The kmer length is longer than nucleotide sequence")
            skipped_counts += 1

    return output_dict

In [16]:
def bar_interactive(length):
    sequence = read_sequence(filename)
    output = kmer_frequency(sequence,length)
    top10 = dict(Counter(output).most_common(10))
    
    #Plot the bar chart for top 10 values
    fig1 = plt.figure(figsize = (10,5))
    ax = fig1.add_subplot(1,1,1)
    ax.bar(list(top10.keys()), top10.values(), color = "b")
    ax.set_xlabel("Fragments",fontsize = 15)
    ax.set_ylabel("Counts",fontsize = 15)

In [19]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF170YQV_rep2_1.fq'
sequence = read_sequence(filename)
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [8]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF239CML_rep1_1.fq'
sequence = read_sequence(filename)
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [9]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF239CML_rep1_2.fq'
sequence = read_sequence(filename)
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [10]:
%matplotlib inline
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF170YQV_rep2_2.fq'
sequence = read_sequence(filename)
length = 6

# create interactive sliders for kmer sequence
length_widget = IntSlider(min = 1, max = 1000, value = length)

# adjust settings to prevent continous recalculation and update of plot while user drags widget
for item in [ length_widget ]:
    item.continuous_update = False

interactive(bar_interactive, length = length_widget)

interactive(children=(IntSlider(value=6, continuous_update=False, description='length', max=1000, min=1), Outp…

In [25]:
def shuffled_sequence(sequence, kmer_sequence):
    random.seed(100)
    kmer_counts = []
    
    for i in range(0,100):
        count = 0

        for line in sequence:
            shuffled_line = ''.join(random.sample(line,len(line)))

            if len(line) >= length:
                for i in range(0,len(shuffled_line)-length+1):
                    count += shuffled_line.count(kmer_sequence)

            else:
                print("Warning: The kmer length is longer than nucleotide sequence")

        kmer_counts.append(count)
    print(kmer_counts)
    return kmer_counts


In [ ]:
# Generate the counts for the choosen kmer sequence
filename = '/Users/apple/Desktop/Summer_project/dataset/ENCFF170YQV_rep2_2.fq'
sequence = read_sequence(filename)
kmer_sequence = "GGGAGG"
length = 6
kmer_counts = shuffled_sequence(sequence, kmer_sequence)

# Plot the histogram
fig2 = plt.figure(figsize = (10,5))
ax1 = fig2.add_subplot(1,1,1)

_, bins, _ = ax1.hist(kmer_counts,bins = 100, density=1, alpha=0.5,label = "bins")

ax1.set_xlabel("counts for " + kmer_sequence, fontsize = 15)
ax1.set_ylabel("Proportion", fontsize = 15)
ax1.set_title(f"Mean = {mu:.2f}, SD = {sigma:.2f}",fontsize = 20)